# 本单元用于记录LeetCode每日一题

## 问题1：[HTML 实体解析器](https://leetcode.cn/problems/html-entity-parser/description/)
### 题目
字符串替换, 将字符串中的特殊符号, 根据规则转化为指定符号。
### 分析
- 使用正则表达式进行划分, 由于我们发现待替换的字符串中都是“&xxx;”的形式，因此我们使用正则表达式进行分组
- re.split(r"(&[^&;]+;)", text):
    - [^&;]: 表示匹配不包括&和;符号
    - +: 表示前面的匹配至少出现一次
    - [^&;]+: 表示必须匹配一个非&和非;的字符
    - (): 表示将内部匹配到的东西作为一个分组整体返回(保证了分割字符中包含有&;) 
### 优化
- 直接使用replace(目标值, 更换值)进行替换即可


In [ ]:
import re
def process(text):
    word_dict = {"&quot;":"\"","&apos;":"\'","&amp;":"&", "&gt;":">","&lt;":"<","&frasl;":"/"}
    word_list = re.split(r"(&[^&;]+;)", text)
    print(word_list)
    for index, word in enumerate(word_list):
        if word in word_dict:
            word_list[index] = word_dict[word]
    return "".join(word_list)

text = "and I quote: &quot;...&quot;"
process(text)


## 问题2：[确定两个字符串是否接近](https://leetcode.cn/problems/determine-if-two-strings-are-close/description/)
### 题目
如果可以使用以下操作从一个字符串得到另一个字符串，则认为两个字符串接近:
- 交换任意两个现有字符的位置.
- 将某种字符与另一种字符全部互换。
若通过有限步的上述两种操作，可以从原字符串得到另一个字符串则认为两个字符串相近

### 分析
- 由于可以交换任意两个字符位置，因此只要保证原字符串与目标字符串的每种字符个数是一样的即可。
- 又由于可以交换任意两种字符(即把所有的a变成b,同时把所有的b变成a), 因此只用保证原字符串的中字符个数列表与目标字符串字符个数列表具有相同元素即可.(例如[1,2,1] 与 [2,1,1])

### 注意
- 由于两个步骤必须与现有的字符交换，因此如果目标字符串中有字母a，但原字符串中没有字母a那将永远无法转换。

In [ ]:
from collections import Counter

def process(word1, word2):
    ## 得到每个字母计数的数值列表
    count1 = Counter(word1)
    count2 = Counter(word2)

    if set(count1.keys()) != set(count2.keys()):
        return False
    
    count1_list = count1.values()
    count2_list = count2.values()

    ### 判断word的元素个数列表是否相同
    count = {}
    for x in count1_list:
        if x in count:
            count[x] += 1
        else:
            count[x] = 1
    for x in count2_list:
        if x not in count or count[x] == 0:
            return False
        count[x] -= 1

    # 以上代码可以通过以下方式替换(时间复杂度会上升)
    # if sorted(count1_list) != sorted(count2_list):
    #     return False

    return True

word1 = "uau"
word2 = "ssx"
process(word1, word2)


## 问题3：[最小体力消耗路径(并查集)](https://leetcode.cn/problems/path-with-minimum-effort)
### 题目
一开始你在最左上角的格子 (0, 0) ，且你希望去最右下角的格子 (rows-1, columns-1) （注意下标从 0 开始编号）。你每次可以往 上，下，左，右 四个方向之一移动，你想要找到耗费 体力 最小的一条路径。一条路径耗费的 体力值 是路径上相邻格子之间 高度差绝对值 的 最大值 决定的。

请你返回从左上角走到右下角的最小 体力消耗值 。

### 分析
- 如果使动态规划dp[i][j]表示到i,j位置的最小体力, 先从左到右遍历更新上,左位置，再用右到左遍历更新下，右位置。但实际上这种方式是不对的。因为方向的改变会导致对前面的状态改变，因此不可取。
- 如果把每个一个格子看作是一个地点，那这个题目则是让我们找最短路径，只不过这个最短路径的定义不同。我们可以使用并查集方式，每次放入一个点，当我们加入一条权值为 x 的边之后，如果左上角和右下角从非连通状态变为连通状态，那么 x即为答案(我们先根据权重从小到大排列)。

### 注意
- 联通性判断类型的题目都可以使用并查集


In [ ]:
class UnionFind:
    """"标准并查集对象
    """
    def __init__(self, n: int):
        self.pre = list(range(n))  ## 初始化每一个点都是以自己为主元

    ## 查找函数
    def find(self, x):
        if self.pre[x] == x:
            return x
        else:
            self.pre[x] = self.find(self.pre[x])
            return self.pre[x]

    ## 并函数
    def join(self, x, y):
        fx = self.find(x)
        fy = self.find(y)
        if fx != fy:
            self.pre[fx] = fy # 定义fx的上级为fy

    ## 连通性判断x,y是否联通
    def connected(self, x: int, y: int) -> bool:
        x, y = self.find(x), self.find(y)
        return x == y


def process(heights):
    m, n = len(heights), len(heights[0])
    edges = []  # 构造边对象
    for i in range(m):
        for j in range(n):
            iden = i * n + j # 当前的坐标拉平后的位置
            if i > 0:  ## 由于是双向关系，因此只用保证上下，左右关系即可
                edges.append((iden - n, iden, abs(heights[i][j] - heights[i - 1][j])))
            if j > 0:
                edges.append((iden - 1, iden, abs(heights[i][j] - heights[i][j - 1])))
    edges = sorted(edges, key=lambda x: x[2])
    uf = UnionFind(m * n)  # 初始化并查集
    ans = 0

    for x, y, v in edges:
        uf.join(x, y)
        if uf.connected(0, m * n - 1):
            ans = v
            break
    return ans

## 问题4：[猜数字游戏](https://leetcode.cn/problems/bulls-and-cows/)
### 题目
给你一个秘密数字 secret 和朋友猜测的数字 guess ，请你返回对朋友这次猜测的提示。提示的格式为 "xAyB" ，x 是公牛个数， y 是奶牛个数，A 表示公牛，B 表示奶牛。请注意秘密数字和朋友猜测的数字都可能含有重复数字。
- 猜测数字中有多少位属于数字和确切位置都猜对了（称为 "Bulls"，公牛），
- 有多少位属于数字猜对了但是位置不对（称为 "Cows"，奶牛）。也就是说，这次猜测中有多少位非公牛数字可以通过重新排列转换成公牛数字。

### 分析
- 维护两个数组true_num和guess_num, true_num[i]表示数字i在secret中出现的次数，guess_num[i]表示数字i在guess中出现的次数
- 同时遍历secret 和 guess的i位置, 如果secret[i] == guess[i]则公牛数字加1, 否则
    - 先进行判断, 当前的正确数字是否已经出现在了guess_num中, 如果是则奶牛数字加1, 同时guess_num对应位置数字出现次数减一
    - 当前的正确数字还没出现在guess_num中，则true_num对应位置+1
    - 再判断猜测数字，是否已经出现在了true_num中, 如果是则奶牛数字加1, 同时true_num对应位置数字出现次数减一
    - 当前的猜测数字还没出现在true_num中，则guess_num对应位置+1


In [ ]:
def process(secret, guess):
    true_nums = [0 for _ in range(10)]
    guess_nums = [0 for _ in range(10)]
    bulls = 0   # 公牛结果
    cows = 0    # 奶牛结果
    for true_num, guess_num in  zip(secret, guess):
        # 判断是否为公牛数字
        if true_num == guess_num:
            bulls += 1
            continue
        
        # 判断是否为奶牛数字
        if guess_nums[int(true_num)] != 0:
            cows += 1
            guess_nums[int(true_num)] -= 1
        else:
            true_nums[int(true_num)] += 1
        
        # 继续判断是否为奶牛数字
        if true_nums[int(guess_num)] != 0:
            cows += 1
            true_nums[int(guess_num)] -= 1
        else:
            guess_nums[int(guess_num)] += 1
        
    return f"{bulls}A{cows}B"

secret = "1123"
guess = "0111"
process(secret, guess)

## 问题5：[需要添加的硬币的最小数量](https://leetcode.cn/problems/minimum-number-of-coins-to-be-added/description/)
### 题目
给你整数数组coins, 表示可用的硬币的面值, 以及一个整数target。如果存在某个coins的子序列总和为x，那么整数x就是一个可取得的金额。我们需要往coins中添加数字, 使得对于[1, target]的每一个值都是可取得的(即,都等于coins中某一个子序列的总和)。请返回最少需要添加多少个数字。

### 分析
- 我们需要将coins进行排序后遍历, 从最小值开始, 如果我们可以得到[1, x]中任意数都可以取得。
    - 此时如果下一个硬币比x大且不是x+1这个数值, 那我们将永远无法得到x+1(因为后面的coin都比x+1大, 意味着这个x+1是无法被取得), 因此我们必须添加x+1这个硬币。当我们添加了x+1这个硬币, 此时我们可以得到的取值范围就变成了[x+1, x+x+1], 且[1, x]我们依然可以取得到, 因此添加完x+1这个硬币后, 我们可以取得的范围为[1, x+x+1]
    - 如果这个值比x小(假设为s), 意味着我们可以不用添加硬币, 而还可以扩大取值范围依到[1, x+s]
- 一直循环直到我们的范围覆盖了[1, target]
- 有可能我们循环了完了coins都没能达到target, 我们还得继续添加, 因此使用了一个技巧, 在while中使用index来控制是否前进数组。如果我们添加硬币, 则还需要判定当前的coin, 因此我们不用增加index, 从而实现重复判定。


In [ ]:
def process(coins, target):
    coins.sort()
    ans = 0
    max_coin = 0  # 初始为0
    index = 0
    while max_coin < target:
        if index < len(coins) and coins[index] <= max_coin+1:   # 如果小于等于最大范围+1, 我们都可以不用添加硬币, 从而增大范围
            max_coin += coins[index]
            index += 1
        else:          # 如果当前硬币大于了最大范围, 我们需要添加一个max_coin+1元素
            ans += 1  
            max_coin = max_coin*2 + 1   # 注意我们此时并不增加index, 继续让其判定当前的coin

    return ans

coins = [1,4,10]
target = 19
process(coins, target)

## 问题6：[访问完所有房间的第一天](https://leetcode.cn/problems/first-day-where-you-have-been-in-all-the-rooms/description/)
### 题目
给你一个长度为n数组nextVisit(访问房间号)。在接下来的几天中，你访问房间的次序将根据下面的规则决定：假设某一天，你访问i号房间。
- 如果算上本次访问，访问 i 号房间的次数为**奇数**，那么第二天需要访问第nextVisit[i]号房间 (数据会保证 0<= nextVisit[i] <= i)。
- 如果算上本次访问，访问 i 号房间的次数为**偶数**，那么第二需要访问(i + 1) mod n 号房间 (访问下一间, 如果i为最后一间, 则从头访问第0间)。

请返回你访问完所有房间日期编号(也就是多少天后, 你第一次把所有的房间都访问完).

### 分析
- 注意到关键点 nextVisit[i] <= i, 意味着当奇数次访问第i号房间时, 一定会回访前面的房间。因此当我访问到第i号房间时, 对于i左边的房间, 我们一定都访问了偶数次(不然不可能到达i, 可以反证)。
- 当我们从i号房间需要回访到j号房间时, 此时[j, i−1]范围内的房间都处于访问偶数次的状态。那么当我们访问这个范围内的每个房间时，算上本次访问，访问次数一定是奇数。所以要想重新回到i, 对于[j, i−1]范围内的每个房间，我们都需要执行一次「回访」。
- f[i] 表示从 「访问到房间i且次数为奇数」 到 「访问到房间i且次数为偶数」 所需要的天数。这个天数是固定的, 只有经历过这个天数后, 才能访问到下一间i+1号房间。注意: 首次访问房间i的一天，和重新回到房间i的一天, 因此状态转移时+2天
- 状态转移方程:f[i] = 2 + sum(f[j], f[2]...f[i-1]), 初始状态f[0]=0, 这里的j为nextVisit[i]
- 我们最后的答案是: sum(f[0], f[1], ..., f[n-1]) + 1; 这里加1是访问第n间的那天
- 由于我们这里需要多次求和, 因此我们定义f[n]的前n-1项和（前缀和）s[n] = sum(f[0], f[1], ..., f[n-1]), 状态转移方程就变为了: f[n] = s[n] - s[n-1] + 2。 我们的最终结果即求: s[n] + 1
- 最后的转移方程为: s[i+1] = s[i] * 2 - s[j] + 2

In [ ]:
def process(nextVisit):
    s = [0] * len(nextVisit)
    for index, num in enumerate(nextVisit[:-1]):
        s[index+1] = s[index] * 2 - s[num] + 2
    return s[-1]+1-1    # 我们求的是日期, 我们从第0天开始的(第0天的访问算天数的), s[-1]+1是经过了多少天

nextVisit = [0,0]
process(nextVisit)

## 问题7：[验证二叉树的前序序列化](https://leetcode.cn/problems/verify-preorder-serialization-of-a-binary-tree/description/)
### 题目
给定一串以逗号分隔的序列("9,3,4,#,#,1,#,#,2,#,6,#,#")，验证它是否是正确的二叉树的**前序序列化**。编写一个在不重构树的条件下的可行算法。注意不允许重建树

### 分析
- 我们知道在树（甚至图）中，所有节点的入度(该节点有几个父节点)之和等于出度(该节点有几个子节点)之和。
- 每个非#节点, 会提供总入度+1, 总出度+2; 每个#节点, 会提供总入度+1, 总出度+0; 注意根节点只提供2个出度
- 遍历整个序列, 如果总入度和总出度的差小于0则说明该二叉数无效。注意在遍历中如果再加入某个节点前已经出现了入度 == 出度, 说明不可能再能加入新的节点(因为入度==出度意味着每个叶节点都是null了)

### 方案二: 模拟递归栈
- 把有效的叶子节点使用 "#" 代替。 比如把 4## 替换成 # 。此时，叶子节点会变成空节点！
- 先把节点压入栈中, 检查栈顶元素是否满(数字 # #)的结构, 如果时则抛出栈顶三元素, 把#压入栈顶(即把 4## 替换成 #), 循环上述操作, 直到无法压缩, 再遍历下一个节点。
- 最后判断是否只剩下["#"]即可

In [ ]:
def process(preorder):
    nodes = preorder.split(",")

    if nodes[0] == "#" and len(nodes) > 1:  # 根节点为空, 直接返回
        return False

    rudu = -1    # 根节点本是不增加入度的
    chudu = 0    
    for node in nodes:
        if rudu == chudu:   # 如果入度等于出度, 说明在进入这个节点之前, 已经达到了是完整二叉树了, 不能再加节点了
            return False
        
        rudu += 1           # 无论是否为空节点, 都会增加入度
        if node != "#":
            chudu += 2
        
    return chudu-rudu == 0

preorder = "7,2,#,2,#,#,#,6,#"
process(preorder)

In [ ]:
# 方案二使用栈
def process(preorder):
    stack = []
    for node in preorder.split(','):
        stack.append(node)
        while len(stack) >= 3 and stack[-1] == stack[-2] == '#' and stack[-3] != '#':
            stack.pop(), stack.pop(), stack.pop()
            stack.append('#')
    return len(stack) == 1 and stack.pop() == '#'


## 问题8：[统计将重叠区间合并成组的方案数](https://leetcode.cn/problems/count-ways-to-group-overlapping-ranges/description/)
### 题目
给你一个二维整数数组ranges，其中ranges[i] = [start, end]表示 start 到 end之间(包括两端点)的所有整数都包含在第i个区间中。请返回将ranges分为两个组, 满足以下规则: 
- 如果ranges[i]与ranges[j]有重叠, 则两个区间必须在一个组内。
- 每个ranges[i]只能在一个组中。
- 组可以没有元素

请返回 组的方案数(有多少中划分)

### 分析
- 我们可以将区间进行整合，看能得到多少个互不重叠的整合区间。先将区间排序, 并遍历每个区间, 如果当前区间和**整合区间**有重叠, 则更新整合区间左右边界(只用更新右边界), 如果没有重叠, 则说明当前区间为下一个整合区间的起始区间。
- 遍历结束后, 返回整和区间个数, 分为两组的情况数为(2**整合区间个数)


In [ ]:
def process(ranges):
    ranges.sort(key=lambda x: x[0])
    left = ranges[0][0]
    right = ranges[0][1]
    count = 1
    for x, y in ranges:
        print(x,y)
        if left <= x <= right:
            right = max(right, y)  # 合并区间
        else:
            count += 1  # 整合区间数 +1
            left = x    # 重置整合区间的左右边界
            right = y
    return pow(2, count, 1_000_000_007)

ranges = [[6,10],[5,15]] 
process(ranges)

## 问题9：[]()
### 题目


### 分析


## 问题10：[]()
### 题目


### 分析


## 问题11：[]()
### 题目


### 分析
